In [4]:
import pytesseract
from PIL import Image
import os

png_directory = '/Users/oscarmoxon/Desktop/AI Projects/hackathon/data_files/PURSGLOVE'
text_storage_directory = '/Users/oscarmoxon/Desktop/AI Projects/hackathon/processed_files'

def extract_text_from_png(png_path):
    try:
        text = pytesseract.image_to_string(Image.open(png_path))
        return text
    except Exception as e:
        print(f"Failed to extract text from {png_path}: {str(e)}")
        return None

def extract_and_store_text_from_pngs(directory, storage_directory):
    text_store = {}
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            path = os.path.join(directory, filename)
            text = extract_text_from_png(path)
            if text:
                text_store[filename] = text
                # Store each document's text in a separate file
                with open(os.path.join(storage_directory, f"{filename}.txt"), 'w', encoding='utf-8') as f:
                    f.write(text)

# Example usage
extract_and_store_text_from_pngs(png_directory, text_storage_directory)

ModuleNotFoundError: No module named 'pytesseract'

In [3]:
from pdfminer.high_level import extract_text
import openai
import os
from dotenv import load_dotenv
import pandas as pd
import pickle
from scipy.spatial.distance import cosine
import requests
import json

# Load environment variables from .env file
load_dotenv()

ImportError: cannot import name 'extract_text' from 'pdfminer.high_level' (/Users/oscarmoxon/.pyenv/versions/3.12.2/lib/python3.12/site-packages/pdfminer/high_level.py)

In [1]:
# Access the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

NameError: name 'os' is not defined

In [8]:
# Set the OpenAI API key
openai.api_key = openai_api_key

# Define the file path with spaces
file_path = "/Users/oscarmoxon/Desktop/AI Projects/hackathon/SmartRedBox/data_files"
directory_path = "/Users/oscarmoxon/Desktop/AI Projects/hackathon/SmartRedBox/processed_files"

client = openai.Client()
print(client)

In [24]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    # Implement your PDF text extraction logic here
    return extract_text(pdf_path)

# Function to get embeddings from text
def get_embeddings(text, model="text-embedding-ada-002"):
    response = openai.Embedding.create(
        model=model,
        input=[text]  # Input needs to be a list for the embeddings API
    )
    return response['data'][0]['embedding']

def extract_and_store_text_from_pdfs(directory, storage_directory):
    text_store = {}
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            try:
                text = extract_text(path)
                text_store[filename] = text
                # Store each document's text in a separate file
                with open(os.path.join(storage_directory, f"{filename}.txt"), 'w', encoding='utf-8') as f:
                    f.write(text)
            except Exception as e:
                print(f"Failed to process {filename}: {str(e)}")

output = extract_and_store_text_from_pdfs(file_path, directory_path)

In [28]:
embeddings_store = {}

def get_embeddings(text, model="text-embedding-ada-002"):
    """
    Fetches embeddings for the provided text using the specified model.
    This function is adjusted to comply with openai API version >= 1.0.0.
    """
    try:
        # Ensure that only the first 2048 characters are sent if the model's limit is 2048 tokens
        response = client.embeddings.create(model=model, input=text[:2048])
        # Correctly accessing the embedding
        # print("Response type:", type(response))
        # print("Response data:", response.data)
        # print("Response data type:", dir(response))
        
        if response.data and len(response.data) > 0:
            embedding_vector = response.data[0].embedding  # Accessing the first embedding in the data array
            return embedding_vector
        else:
            print("No embedding data found in the response.")
            return None

    except Exception as e:
        print(f"Error while fetching embeddings: {e}")
        return None

def embed_and_store_texts(storage_directory, embedding_storage_path):
    embeddings_store = {}
    for filename in os.listdir(storage_directory):
        if filename.endswith(".txt"):
            with open(os.path.join(storage_directory, filename), 'r', encoding='utf-8') as file:
                text = file.read()
                try:
                    embedding = get_embeddings(text)
                    embeddings_store[filename] = embedding
                except Exception as e:
                    print(f"Failed to generate embedding for {filename}: {str(e)}")
    
    # Save embeddings to disk
    with open(embedding_storage_path, 'wb') as f:
        pickle.dump(embeddings_store, f)

embeddings = embed_and_store_texts(directory_path, 'embeddings_store.pkl')

Response type: <class 'openai.types.create_embedding_response.CreateEmbeddingResponse'>
Response data: [Embedding(embedding=[-0.014143422245979309, -0.0028018036391586065, -0.03098870813846588, -0.017176125198602676, -0.018926821649074554, 0.011524270288646221, -0.017010705545544624, -0.00971154123544693, -0.035703182220458984, -0.004614532925188541, 0.0037667546421289444, 0.026329373940825462, -0.008505352772772312, -0.0008236544672399759, -0.025598768144845963, 0.007960844784975052, 0.023655081167817116, -0.010738524608314037, 0.00949098076671362, -0.011524270288646221, 0.01971256732940674, 0.006148115266114473, -0.002739771269261837, 0.0014672422548756003, 0.016542013734579086, 0.010131984017789364, 0.007285378873348236, -0.035785891115665436, -0.024289190769195557, -0.002603644272312522, -0.01787916012108326, 0.00023326824884861708, -0.008202082477509975, -0.012751135975122452, -0.005868969019502401, -0.03564804047346115, -0.020360462367534637, -0.009077430702745914, 0.006330766715

In [29]:
def load_embeddings(embedding_storage_path):
    with open(embedding_storage_path, 'rb') as f:
        embeddings_store = pickle.load(f)
    return embeddings_store

embeddings_store = load_embeddings('embeddings_store.pkl')

In [30]:
print(embeddings_store)

{'Written-Questions-Answers-Statements-Daily-Report-Commons-2024-04-12.pdf.txt': [-0.014143422245979309, -0.0028018036391586065, -0.03098870813846588, -0.017176125198602676, -0.018926821649074554, 0.011524270288646221, -0.017010705545544624, -0.00971154123544693, -0.035703182220458984, -0.004614532925188541, 0.0037667546421289444, 0.026329373940825462, -0.008505352772772312, -0.0008236544672399759, -0.025598768144845963, 0.007960844784975052, 0.023655081167817116, -0.010738524608314037, 0.00949098076671362, -0.011524270288646221, 0.01971256732940674, 0.006148115266114473, -0.002739771269261837, 0.0014672422548756003, 0.016542013734579086, 0.010131984017789364, 0.007285378873348236, -0.035785891115665436, -0.024289190769195557, -0.002603644272312522, -0.01787916012108326, 0.00023326824884861708, -0.008202082477509975, -0.012751135975122452, -0.005868969019502401, -0.03564804047346115, -0.020360462367534637, -0.009077430702745914, 0.006330766715109348, 0.01131060253828764, -0.00234862137

In [31]:
from scipy.spatial.distance import cosine

def find_most_similar(text_embedding, embeddings_store, top_k=5):
    similarities = {}
    for filename, embedding in embeddings_store.items():
        sim = 1 - cosine(text_embedding, embedding)  # cosine returns the distance, so subtract from 1
        similarities[filename] = sim

    # Sort by similarity score in descending order
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return sorted_similarities[:top_k]

def get_query_embedding(text, model="text-embedding-ada-002"):
    # You would need the client set up and the get_embeddings function defined previously
    return get_embeddings(text, model=model)

# Example usage: Assuming `client` is your OpenAI client setup and `get_embeddings` is defined
query_text = "Enter some example text here that you want to find similar documents for."
query_embedding = get_query_embedding(query_text)

# Assuming embeddings_store is loaded
most_similar_docs = find_most_similar(query_embedding, embeddings_store)
for doc, similarity in most_similar_docs:
    print(f"Document: {doc}, Similarity: {similarity}")

Response type: <class 'openai.types.create_embedding_response.CreateEmbeddingResponse'>
Response data: [Embedding(embedding=[-0.011876553297042847, -0.0022789589129388332, -0.001294940710067749, -0.004814172629266977, 0.0012394188670441508, 0.024450119584798813, -0.018819348886609077, -0.009225171059370041, -0.02431345172226429, -0.007653475273400545, 0.01165788248181343, 0.013441415503621101, -0.0009250796283595264, 0.013837756589055061, -0.006515704095363617, 0.008425655774772167, 0.025967149063944817, -0.0045476676896214485, 0.017343321815133095, -0.027948850765824318, -0.015170280821621418, -0.018682679161429405, -0.002866636496037245, -0.0031485166400671005, -0.020268041640520096, -0.0070589641109108925, 0.00984701607376337, -0.021853405982255936, -0.020568715408444405, -0.015088279731571674, 0.006843710318207741, -0.014104261063039303, -0.012894739396870136, 0.004991842433810234, 0.008671660907566547, 0.0031211827881634235, 0.008774162270128727, -0.01955736242234707, 0.0428867936

In [55]:
import re

def load_document_text(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def extract_excerpt(text, max_sentences=3):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text, max_sentences)
    excerpt = ' '.join(sentences[:max_sentences])
    return excerpt

def serve_results(similar_docs, num_docs = 2):
    fetched = {}
    for filename, _ in similar_docs[:num_docs]:
        full_path = os.path.join(directory_path, filename)
        text = load_document_text(full_path)
        # Only show the first 1000 characters
        # print("Loaded document:", text[:100], "...")
        
        excerpt = extract_excerpt(text)
        fetched[filename] = excerpt[:200]  # store first 200 characters of the excerpt

        # print("Document:", filename)
        # print("Extracted excerpt:", excerpt[:200], "...")
        # print("\n---\n")

    return fetched

excerpts = serve_results(most_similar_docs)

for filename, excerpt in excerpts.items():
    print(f"Document: {filename}, Excerpt: {excerpt[:200]} ...\n")

Document: Written-Questions-Answers-Statements-Daily-Report-Commons-2024-03-22.pdf.txt, Excerpt:  Daily Report 

Friday, 22 March 2024 

This report shows written answers and statements provided on 22 March 2024 and the 
information is correct at the time of publication (03:31 P.M., 22 March 2024 ...

Document: Written-Questions-Answers-Statements-Daily-Report-Commons-2024-03-26.pdf.txt, Excerpt:  Daily Report 

Tuesday, 26 March 2024 

This report shows written answers and statements provided on 26 March 2024 and the 
information is correct at the time of publication (07:53 P.M., 26 March 202 ...

